##### <a id='toc1_1_1_1_1_'></a>[Setting up data and imports](#toc0_)


In [1]:
import numpy as np
from importlib import reload

# Importing my modules
from src.main import (
    classes,
    geometry_operations,
    optimization_functions,
    plotting_2d,
    plotting_3d,
)

In [2]:
# %store -r line_gdf_reworked
%store -r line_gdf_75
%store -r harvesteable_trees_gdf
%store -r height_gdf
%store -r anchor_trees_gdf
%store -r target_trees_gdf
%store -r line_gdf_v1
%store -r slope_line
%store -r results_df 
#rewrite this if the result of the main optimization changed

line_gdf = line_gdf_v1.copy()
# hack to extract this from the line_gdf
start_point_dict = dict(
    [(key, value.coords[0]) for key, value in enumerate(line_gdf["line_candidates"])]
)

#### <a id='toc1_1_1_2_'></a>[Optimization Setup](#toc0_)


In [3]:
bhd_series = harvesteable_trees_gdf["BHD"]
height_series = harvesteable_trees_gdf["h"].replace(",", ".", regex=True).astype(float)

# Prepare the gdfs
uphill_yarding = False
line_gdf["line_cost"] = optimization_functions.compute_line_costs(
    line_gdf, uphill_yarding, large_yarder=True
)
harvesteable_trees_gdf["cubic_volume"] = optimization_functions.compute_tree_volume(
    bhd_series, height_series
)

In [4]:
line_gdf = line_gdf.iloc[0:10]
harvesteable_trees_gdf = harvesteable_trees_gdf.iloc[0:100]

#### <a id='toc1_1_1_3_'></a>[Single-Objective](#toc0_)


In [4]:
reload(optimization_functions)
reload(classes)
# Lists to store the results
optimization_result_list = []
lscp_model_list = []

for i in range(0, 3):
    print(f"Starting with objective {i}")
    # set up the model with firs the monetary objective (0), then sideways slope (1) and steep segments (2) as single objective
    lscp_optimization = classes.optimization_model(
        "Single Objective",
        line_gdf,
        harvesteable_trees_gdf,
        height_gdf,
        slope_line,
        objective_to_select=i,
    )
    lscp_optimization.add_generic_vars_and_constraints()
    lscp_optimization.add_single_objective()
    # and solve it
    lscp_optimization.solve()
    lscp_model_list.append(lscp_optimization)

Starting with objective 0
Starting with objective 1
Starting with objective 2


In [5]:
for count, optimization_object in enumerate(lscp_model_list):
    optimization_result_list.append(
        classes.optimization_result(
            optimization_object,
            line_gdf,
            0,
            False,
            optimization_object.name + str(count),
        )
    )

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodat

#### <a id='toc1_1_1_4_'></a>[NSGA 2](#toc0_)


In [6]:
from src.main import moo_optimization_functions
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

reload(moo_optimization_functions)

sideway_lines = lscp_model_list[0].sideways_slope_deviations_per_cable_road
steep_lines = lscp_model_list[0].steep_downhill_segments

cost_matrix = lscp_model_list[0].aij
nsga_problem = moo_optimization_functions.SupportLinesProblem(
    cost_matrix, lscp_model_list[0].facility_cost, sideway_lines, steep_lines
)
termination = get_termination("n_gen", 20)

reload(optimization_functions)

client_range = cost_matrix.shape[0]
facility_range = cost_matrix.shape[1]

algorithm = NSGA2(
    pop_size=10,
    sampling=moo_optimization_functions.CustomSampling(),  # initally zero matrix, nothing assigned
    mutation=moo_optimization_functions.MyMutation(),
    repair=moo_optimization_functions.MyRepair(),
)

# %prun minimize(problem,algorithm,termination,verbose=True,return_least_infeasible=True,seed=0)
res = minimize(
    nsga_problem,
    algorithm,
    termination,
    verbose=True,
    return_least_infeasible=True,
    seed=0,
)

X = res.X
F = res.F

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        1 |      1 |  5.159484E+02 |  5.159484E+02 |             - |             -
     2 |       11 |      7 |  0.000000E+00 |  0.000000E+00 |             - |             -
     3 |       21 |      8 |  0.000000E+00 |  0.000000E+00 |  0.1049388002 |         nadir
     4 |       31 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0118200212 |         ideal
     5 |       41 |     10 |  0.000000E+00 |  0.000000E+00 |  0.0877497869 |         ideal
     6 |       51 |     10 |  0.000000E+00 |  0.000000E+00 |  0.0160540052 |             f
     7 |       61 |     10 |  0.000000E+00 |  0.000000E+00 |  0.2220232509 |         ideal
     8 |       71 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0138775925 |         ideal
     9 |       81 |     10 |  0.000000E+00 |  0.000000E+00 |  0.0063701220 |             f
    10 |       91 |     10 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

In [35]:
reload(classes)
len_x = len(res.X)
samples = 3
for i in np.linspace(0, len_x - 1, samples).astype(int):
    optimization_result_list.append(
        classes.optimization_result(res, line_gdf, i, False, "NSGA2 " + str(i))
    )

/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodat

In [41]:
reload(classes)
reload(plotting_2d)

tree_volumes_list = harvesteable_trees_gdf["cubic_volume"]
(
    distance_tree_line,
    distance_carriage_support,
) = geometry_operations.compute_distances_facilities_clients(
    harvesteable_trees_gdf, line_gdf
)
sample_model = lscp_model_list[0]

results_df = plotting_2d.model_results_comparison(
    optimization_result_list,
    line_gdf,
    sample_model.aij,
    distance_carriage_support,
    sample_model.productivity_cost,
    tree_volumes_list,
    sample_model.sideways_slope_deviations_per_cable_road,
    sample_model.steep_downhill_segments,
)

Profit baseline is 51853.884670017214


### AUGMECON MOO


#### Creating the reference table for AUGMECON approach


In [47]:
# determine the ranges of the objectives and divide them in 10 equal parts
sideways_true_max = results_df["sideways_slope_deviations"].max()
sideways_true_min = results_df["sideways_slope_deviations"].min()
# sideways_true_min = 120
downhill_true_max = results_df["steep_downhill_segments"].max()
downhill_true_min = results_df["steep_downhill_segments"].min()
# downhill_true_min = 63

# first determine the ranges of the objectives
max_overall_profit = results_df["overall_profit"].max()
min_overall_profit = results_df["overall_profit"].min()

# create a grid of points to evaluate the objective functions
profit_range = np.linspace(min_overall_profit, max_overall_profit, 3)

sideways_range, sideways_step = np.linspace(
    sideways_true_max,
    sideways_true_min,
    3,
    retstep=True,
)

downhill_range, downhill_step = np.linspace(
    downhill_true_max, downhill_true_min, 3, retstep=True
)

In [48]:
from itertools import islice

reload(classes)
reload(optimization_functions)
# line_gdf = line_gdf.iloc[0:5]
# harvesteable_trees_gdf = harvesteable_trees_gdf.iloc[0:20]
pareto_optimal_objective_values = [0]
pareto_optimal_objects = []


def augmecon():
    initial_model = classes.optimization_model(
        "Single Objective",
        line_gdf,
        harvesteable_trees_gdf,
        height_gdf,
        slope_line,
        objective_to_select=0,
    )

    initial_model.add_generic_vars_and_constraints()
    # add the main monetary objective
    initial_model.add_single_objective()
    initial_model.solve()

    # set up the ranges at iteration objects so we can skip steps in the loop
    i_range = iter(sideways_range)

    for i in i_range:
        print("i should be :", i)
        initial_model.add_epsilon_constraint(target_value=i, objective_to_select=1)

        try:
            initial_model.solve()
        except:
            print("couldnt solve with i ", i)
            break

        (
            cost_objective,
            sideways_objective,
            steep_objective,
        ) = initial_model.get_objective_values()
        # determine the slack variable of the sideways constraint - this is the value of the objective function minus the expected value as per the sideways range
        i_slack = sideways_objective - i
        print("i is :", sideways_objective)

        j_range = iter(downhill_range)
        # loop through the inner objective
        for j in j_range:
            print("          j should be:", j)
            initial_model.add_epsilon_constraint(target_value=j, objective_to_select=2)

            try:
                initial_model.solve()
            except:
                print("couldnt solve with j ", j)
                break

            # determine the slack variable of the downhill constraint - this is the value of the objective function minus the expected value as per the downhill range
            j_slack = steep_objective - j
            print("          j is : ", steep_objective)
            print("          cost is :", cost_objective)

            # since we could solve this problem, this is a pareto-optimal solution - get the objective value and store it
            overall_objective = initial_model.get_total_epsilon_objective_value(
                (sideways_true_max - sideways_true_min),
                (downhill_true_max - downhill_true_min),
            )

            result = classes.optimization_result(
                initial_model, line_gdf, 0, False, "Augmecon"
            )

            # pareto_optimal_objective_values.append(overall_objective)
            pareto_optimal_objects.append(result)

            (
                cost_objective,
                sideways_objective,
                steep_objective,
            ) = initial_model.get_objective_values()

            # set the new objective
            initial_model.add_epsilon_objective(
                i_slack, j_slack, sideways_range, downhill_range
            )

            surface_plot_data_x.append(cost_objective)
            surface_plot_data_y.append(sideways_objective)
            surface_plot_data_z.append(steep_objective)

            if j_slack > 0:
                print("couldnt improve objective?")
                break  # skipping the rest of the downhill range since we cant improve the objective anymore
            else:
                # the expected value as per the downhill range. If the slack variable is greater than what we would constrain for the next step, we skip those iterations
                j_bypass = int(abs(np.floor(j_slack / downhill_step)))
                if j_bypass > 0:
                    # for iterator j, skip j_bypass steps
                    print("         skipping j_bypass:", j_bypass)
                    next(islice(j_range, j_bypass, j_bypass), None)


surface_plot_data_x = []
surface_plot_data_y = []
surface_plot_data_z = []
augmecon()

i should be : 396.23903644902464
i is : 384.9645946343182
          j should be: 396.23903644902464
          j is :  326.74400678183684
          cost is : 5336.204000281543


/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


          j should be: 227.30915875479906
          j is :  326.74400678183684
          cost is : 5336.204000281543


/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


couldnt improve objective?
i should be : 255.53501873981452
i is : 250.7978966654938
          j should be: 396.23903644902464
          j is :  195.96690153064242
          cost is : 6312.995086885745


/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


         skipping j_bypass: 1
          j should be: 58.3792810605735
          j is :  253.56771465575008
          cost is : 6249.614268268009


/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/carlo/GitHub/MultiObjectOptimizationCableYarding/moo_env/lib/python3.10/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


couldnt improve objective?
i should be : 114.83100103060437
couldnt solve with i  114.83100103060437


In [57]:
optimization_result_list = optimization_result_list + pareto_optimal_objects

In [ ]:
a = []

#### Comparison of Optimization Results


In [1]:
tree_volumes_list = harvesteable_trees_gdf["cubic_volume"]
(
    distance_tree_line,
    distance_carriage_support,
) = geometry_operations.compute_distances_facilities_clients(
    harvesteable_trees_gdf, line_gdf
)
sample_model = lscp_model_list[0]
reload(classes)
reload(plotting_2d)
import pandas as pd

pd.options.plotting.backend = "hvplot"

results_df = plotting_2d.model_results_comparison(
    optimization_result_list,
    line_gdf,
    sample_model.aij,
    distance_carriage_support,
    sample_model.productivity_cost,
    tree_volumes_list,
    sample_model.sideways_slope_deviations_per_cable_road,
    sample_model.steep_downhill_segments,
)

results_df[results_df.columns[:6]] = results_df[results_df.columns[:6]].astype(int)
results_df[results_df.columns[7:]] = results_df[results_df.columns[7:]].astype(int)
results_df = results_df.reindex(
    columns=[
        "name",
        "Productivity cost per m3 as per Stampfer",
        "overall_profit",
        "profit_comparison",
        "cable_road_costs",
        "Total distance from carriage to support",
        "Total distance of trees to cable roads",
        "sideways_slope_deviations",
        "steep_downhill_segments",
    ]
)
results_df.set_index("name", inplace=True)
display(results_df)
results_df.plot()

NameError: name 'harvesteable_trees_gdf' is not defined

In [67]:
reload(plotting_2d)
for result in optimization_result_list:
    fig = plotting_2d.plot_optimization_layout(result, line_gdf, harvesteable_trees_gdf)
    fig.show("notebook_connected")

In [71]:
results_df.columns

Index(['Total distance of trees to cable roads',
       'Productivity cost per m3 as per Stampfer',
       'Total distance from carriage to support', 'overall_profit',
       'cable_road_costs', 'profit_comparison', 'name',
       'sideways_slope_deviations', 'steep_downhill_segments'],
      dtype='object')

In [74]:
reload(plotting_2d)
import plotly.graph_objects as go
from ipywidgets import VBox

f = go.FigureWidget(
    go.Scatter3d(
        x=results_df["overall_profit"],
        y=results_df["sideways_slope_deviations"],
        z=results_df["steep_downhill_segments"],
    )
)

t = go.FigureWidget([])


def selection_fn(trace, points, selector):
    # get index of this point in the trace
    index = points.point_inds[0]

    # update the layout plot to show this result
    t.data = []

    t.add_traces(
        list(
            plotting_2d.plot_optimization_layout(
                optimization_result_list[index], line_gdf, harvesteable_trees_gdf
            ).data
        )
    )

    t.update_layout(title=optimization_result_list[index].name)


t.update_layout(width=1200, height=800)
f.data[0].on_click(selection_fn)
VBox((f, t))

    'data': [{'type': 'scatter3d',
              'uid': '8fff27e0-32fd-47d0-80ed…

#### Interactive CR Selection


In [6]:
import plotly.graph_objs as go
import plotly.offline as py

import pandas as pd
import numpy as np
from ipywidgets import VBox

from random import random

# create a trace for the trees
xs, ys = zip(*[(row.xy[0][0], row.xy[1][0]) for row in harvesteable_trees_gdf.geometry])
trees = go.Scatter(
    x=xs,
    y=ys,
    mode="markers",
    marker=dict(color="green"),
    name="trees",
)

# Create traces for each line
individual_lines = [
    go.Scatter(
        x=np.asarray(row.geometry.xy[0]) + random(),
        y=np.asarray(row.geometry.xy[1]) + random(),
        mode="lines",
        line=dict(color="lightgrey"),
        name=str(id),
    )
    for id, row in line_gdf.iterrows()
]

# create a figure from all individual scatter lines
f = go.FigureWidget([*individual_lines, trees])
f.update_layout(
    width=1000,
    height=800,
)


# create the onclick function to select new CRs
def selection_fn(trace, points, selector):
    # since the handler is activated for all lines, test if this one has coordinates, ie. is the clicked line
    if points.xs:
        if trace.line.color == "black":
            f.update_traces(line=dict(color="lightgrey"), selector={"name": trace.name})
        elif trace.line.color == "lightgrey":
            # update this trace to turn black
            f.update_traces(line=dict(color="black"), selector={"name": trace.name})

        # get all active traces
        active_traces = f.select_traces(selector={"line.color": "black"})
        active_traces_names = [int(trace.name) for trace in active_traces]

        # set the dataframe rows to show only these CRs
        t.data[0].cells.values = [df[df.index.isin(active_traces_names)]]
        # and update the dataframe showing the computed costs
        c.data[0].cells.values = update_layout_cost_df(active_traces_names)


# add the onclick function to all traces
for trace in f.data:
    trace.on_click(selection_fn)

# create a dataframe and push it to a figurewidget to display details about our selected lines
df = line_gdf[["line_cost", "line_length"]]
t = go.FigureWidget([go.Table(header=dict(values=df.columns), cells=dict(values=[df]))])

cost_gdf = pd.DataFrame(
    columns=["Tree_to_line_distance", "Productivity_cost", "Line_cost", "Profit"]
)
c = go.FigureWidget(
    [go.Table(header=dict(values=cost_gdf.columns), cells=dict(values=[cost_gdf]))]
)


def update_layout_cost_df(indices):
    rot_line_gdf = line_gdf[line_gdf.index.isin(indices)]

    # Create a matrix with the distance between every tree and line and the distance between the support (beginning of the CR) and the carriage (cloests point on the CR to the tree)
    (
        distance_tree_line,
        distance_carriage_support,
    ) = geometry_operations.compute_distances_facilities_clients(
        harvesteable_trees_gdf, rot_line_gdf
    )

    # assign all trees to their closest line
    tree_to_line_assignment = np.argmin(distance_tree_line, axis=1)

    # compute the distance of each tree to its assigned line
    distance_trees_to_lines = sum(
        distance_tree_line[range(len(tree_to_line_assignment)), tree_to_line_assignment]
    )

    # compute the productivity cost
    productivity_cost_overall = np.sum(
        lscp_model_list[0].productivity_cost[
            range(len(tree_to_line_assignment)), tree_to_line_assignment
        ]
    )

    line_cost = sum(rot_line_gdf["line_cost"])
    return [
        distance_trees_to_lines,
        productivity_cost_overall,
        line_cost,
        3591541 - productivity_cost_overall,
    ]


# Put everything together in a VBox
VBox((f, t, c))

    'data': [{'line': {'color': 'lightgrey'},
              'mode': 'lines',
   …